**Progetto Machine Learning**
---
**Giovanni D'Agostino, Marco Giorgi**

Fase Preliminare: 
- individuare la GPU Tesla-P100
- montare il drive contenente il codice e i dataset
- selezionare i path opportuni
- configurazione size window e batch 

In [ ]:
########## Tipo GPU allocata #############
!nvidia-smi

Thu Dec  3 13:54:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('Project_ML')
sys.path.append('/content/Project_ML/My Drive/ML/ProveFinali/Dishwasher')
path_main_test = '/content/Project_ML/My Drive/ML/ProveFinali/main_train.csv' #Insert main test
path_dishwasher_test = '/content/Project_ML/My Drive/ML/ProveFinali/dishwasher_train.csv' # Insert dishwasher test
window_size= 999 #999 NB!deve avere dimensione dispari
batch_size= 512  #512


Mounted at Project_ML


# **Load Dataset Dishwasher**



In [ ]:
import pandas as pd
import preprocessing 
import metrics
import generator
import stats_data
import tensorflow as tf
import numpy as np
df_main_test = pd.read_csv(path_main_test)
df_dishwasher_test = pd.read_csv(path_dishwasher_test)
model_dishwasher = tf.keras.models.load_model('/content/Project_ML/My Drive/ML/ProveFinali/Dishwasher/modelloFinaleDishwasher', custom_objects=None)

# **Preprocessing**

In [ ]:
import numpy as np
main_power = preprocessing.load_data(df_main_test)
dishwasher_power= preprocessing.load_data(df_dishwasher_test)
main_power_padded = preprocessing.padding(main_power, window_size)
dishwasher_power_padded = preprocessing.padding(dishwasher_power, window_size)
min_Main = np.min(main_power)
max_Main = np.max(main_power)
min_Dish = np.min(dishwasher_power)
max_Dish = np.max(dishwasher_power)
main_power_nrm = preprocessing.normalizzazione(main_power_padded, min_Main, max_Main)
dishwasher_power_nrm = preprocessing.normalizzazione(dishwasher_power_padded, min_Dish, max_Dish)

# **Test**

In [ ]:
generator = generator.DataGenerator(main_power_nrm, dishwasher_power_nrm, window_len=window_size, batch_len=batch_size)
pred_dishwasher = model_dishwasher.predict(x=generator, steps=generator.__len__(), verbose=1)
pred_dishwasher_reverse= preprocessing.reverse_norm(min_Dish, max_Dish, pred_dishwasher)


In [ ]:
pred_dishwasher_reverse= preprocessing.reverse_norm(pred_dishwasher, min_Dish, max_Dish)
f1=metrics.f1_score( dishwasher_power, pred_dishwasher_reverse)
print("F1:",f1)

precision:  [0.99299306]
recall:  [0.9964652]
F1: [0.9947261]
